In [84]:
from abc import ABCMeta, abstractmethod

import panel as pn
import param
from param import rx, Parameterized

from langchain_openai import ChatOpenAI

from dotenv import load_dotenv


load_dotenv()
pn.extension()

In [34]:
spacer = pn.Spacer(
    height=100,
    styles={"background": "teal"},
)
spacer0 = pn.Spacer(
    height=100,
    styles={"background": "red"},
)

In [35]:
gspec = pn.GridSpec(width=800, height=600)

In [36]:
gspec[0, :] = spacer

In [39]:
gspec[1, :] = spacer0


    (1, 0): Spacer(height=300, styles={'background': 'teal'}, width=800)

The following shows a view of the grid (empty: 0, occupied: 1, overlapping: 2):

[[1]
 [2]]


In [45]:
pn.Column(spacer, pn.Spacer(height=20), spacer0)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e01e7589-7171-4850-87ae-6e5254501dfc': {'version…

In [90]:
clear_chat_button = pn.widgets.Button(name="Clear Chat", width=100)
model_chooser = pn.widgets.Select(name="LLM", options=["Model 1", "Model 2", "Model 3"])
corpus_chooser = pn.widgets.Select(
    name="Corpus", options=["Corpus 1", "Corpus 2", "Corpus 3"]
)

chat_sidebar = pn.Column(
    clear_chat_button, corpus_chooser, model_chooser, align="center"
)

In [91]:
chat_feed = pn.chat.ChatFeed(height=400, width=800)

chat_input = pn.chat.ChatAreaInput(width=600, rows=3)
send_button = pn.widgets.Button(name="Send", width=140, height=60)
toggle_retriever = pn.widgets.Toggle(name="Retriever", value=False, width=60, height=60)
chat_send_row = pn.Row(chat_input, send_button, toggle_retriever)

main_chat = pn.Column(chat_feed, chat_send_row)

In [92]:
chat_tab = pn.Row(sidebar, main_chat)
chat_tab

BokehModel(combine_events=True, render_bundle={'docs_json': {'20a9b3ea-bc99-49aa-99ee-3c9d69449943': {'version…

In [106]:
new_corpus_button = pn.widgets.Button(name="New Corpus", width=100)
corpus_name_box = pn.widgets.TextInput(name="Corpus Name", width=200)

strategy_select = pn.widgets.Select(
    name="Strategy", options=["Strategy 1", "Strategy 2", "Strategy 3"]
)
parameters = pn.Column("## Parameters", styles={"background": "whitesmoke"})

corpus_left = pn.Column(new_corpus_button, corpus_name_box, strategy_select, parameters)
corpus_left

BokehModel(combine_events=True, render_bundle={'docs_json': {'e21f5372-801d-449c-b89a-d365f6e00af6': {'version…

In [103]:
current_corpus_name = pn.pane.Markdown(object="# Current Corpus: None")

file_input = pn.widgets.FileInput(name="Upload File")
file_input_button = pn.widgets.Button(name="Upload File", width=100)
file_input_row = pn.Row(file_input, file_input_button)

site_loader_input = pn.widgets.TextInput(
    name="Site Loader", value="https://www.example.com"
)
site_loader_button = pn.widgets.Button(name="Load Site", width=100)
site_loader_row = pn.Row(site_loader_input, site_loader_button)


corpus_mid = pn.Column(
    current_corpus_name, corpus_chooser, file_input_row, site_loader_row
)
corpus_mid

BokehModel(combine_events=True, render_bundle={'docs_json': {'681b277e-fb44-42a9-b162-b028bf324e3d': {'version…

In [105]:
resource_list_to_process = pn.Column(
    "# Resources to Process", styles={"background": "whitesmoke"}
)
resource_list_process = pn.widgets.Button(name="Process", width=100)

resource_list = pn.Column("# Resources Present", styles={"background": "whitesmoke"})

resource_progress = pn.pane.Markdown(object="## Progress")

corpus_right = pn.Column(
    resource_list_to_process, resource_list_process, resource_list, resource_progress
)
corpus_right

BokehModel(combine_events=True, render_bundle={'docs_json': {'6427310c-e1b1-4253-a325-6dbf07207814': {'version…

In [107]:
corpus_tab = pn.Row(corpus_left, corpus_mid, corpus_right)
corpus_tab

BokehModel(combine_events=True, render_bundle={'docs_json': {'5eb51c56-81c7-452b-bf7d-75dffd9d9e54': {'version…

In [116]:
srv.stop()

AssertionError: Already stopped

In [135]:
pn.config.theme = "default"

In [133]:
serve(
    pn.Tabs(
        ("# Chat", chat_tab),
        ("Corpus Settings", corpus_tab),
        stylesheets=[".bk-tab {font-size: 20px}"],
    )
)

Launching server at http://localhost:35594


In [121]:
srv = None


def serve(obj):
    global srv
    try:
        srv.stop()
    except:
        srv = pn.serve(obj, port=35594)
        return srv

In [ ]:
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")
chat_model

In [180]:
chat_model = ChatOpenAI()
# To get the arguments of the chat model, you can use the 'inspect' module
import inspect

print(inspect.getfullargspec(chat_model.__init__))

FullArgSpec(args=['__pydantic_self__'], varargs=None, varkw='data', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={'return': None, 'data': typing.Any})


In [110]:
from langchain_core.language_models import BaseLanguageModel


class ChatModel(param.Parameterized):
    model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)
    model_args = param.List(item_type=str)

    model_args_defaults = param.Dict(
        default={}, doc="Arguments and their default parameters"
    )

    def __init__(self, **params):
        super().__init__(**params)

        self._set_args_defaults()
        self._validate_params()

    def _set_args_defaults(self):
        for model_arg in self.model_args:
            self.model_args_defaults[model_arg] = (
                inspect.signature(self.model).parameters[model_arg].default
            )
            
    def _validate_params(self):
        for parameter, value in self.param.values().items():
            if value is None:
                raise ValueError(f"Parameter '{parameter}' cannot be None")



In [ ]:
class Corpus(param.Parameterized):
    name = param.String()
    corpus_creation_args = param.Dict(default={})
    corpus_retrieval_args = param.Dict(default={})
    
    # Creation logic
    # Retrieval logic
    # Storage logic
    

In [112]:
class ParameterizedABCMeta(ABCMeta, type(Parameterized)):
    pass

class BaseParameterized(Parameterized, metaclass=ParameterizedABCMeta):
    pass

class BaseRAG(BaseParameterized):
    # add params as necessary for external use
    @abstractmethod
    def create_corpus(self):
        pass

class SimpleRAG(BaseRAG):

    def create_corpus(self):
        pass